In [1]:
import kagglehub

path = kagglehub.dataset_download("techsash/waste-classification-data")
print(path)


Using Colab cache for faster access to the 'waste-classification-data' dataset.
/kaggle/input/waste-classification-data


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 224
batch_size = 32

train_path = path + "/DATASET/TRAIN"
test_path  = path + "/DATASET/TEST"

train_gen = ImageDataGenerator(rescale=1./255)
test_gen  = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

test_data = test_gen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)


Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),

    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [5]:
history = model.fit(
    train_data,
    steps_per_epoch=30,      # FAST
    epochs=1,                # FAST
    validation_data=test_data,
    validation_steps=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.6504 - loss: 1.7342 - val_accuracy: 0.8562 - val_loss: 0.4060


In [6]:
model.save("/content/waste_classification_model.h5")


In [7]:
from google.colab import files
files.download("/content/waste_classification_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from google.colab import files
files.upload()


Saving organic.jpg to organic.jpg


{'organic.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xe1\x00ZExif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x05\x03\x01\x00\x05\x00\x00\x00\x01\x00\x00\x00J\x03\x03\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00Q\x10\x00\x01\x00\x00\x00\x01\x01\x00\x00\x00Q\x11\x00\x04\x00\x00\x00\x01\x00\x00\x0e\xc3Q\x12\x00\x04\x00\x00\x00\x01\x00\x00\x0e\xc3\x00\x00\x00\x00\x00\x01\x86\xa0\x00\x00\xb1\x8f\xff\xdb\x00C\x00\x02\x01\x01\x02\x01\x01\x02\x02\x02\x02\x02\x02\x02\x02\x03\x05\x03\x03\x03\x03\x03\x06\x04\x04\x03\x05\x07\x06\x07\x07\x07\x06\x07\x07\x08\t\x0b\t\x08\x08\n\x08\x07\x07\n\r\n\n\x0b\x0c\x0c\x0c\x0c\x07\t\x0e\x0f\r\x0c\x0e\x0b\x0c\x0c\x0c\xff\xdb\x00C\x01\x02\x02\x02\x03\x03\x03\x06\x03\x03\x06\x0c\x08\x07\x08\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\xff\xc0\x00\x11\x08\x01\xf3\x02\xab\x03\x01"\x00\x02\x11\x0

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model("/content/waste_classification_model.h5")

img = image.load_img("/content/organic.jpg", target_size=(224,224))
img = image.img_to_array(img)/255.0
img = np.expand_dims(img, axis=0)

pred = model.predict(img)
print("Organic (O)" if pred[0][0] < 0.5 else "Recyclable (R)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Organic (O)
